# Construction of the Dependent Variable


In [6]:
# This tells matplotlib not to try opening a new window for each plot.
#%matplotlib inline

# General libraries.
import os
import codecs
import json
import csv

import re
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy
import glob


In [3]:
path =r'/home/nyao111/266/FinalProject/price_history' 
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    df['ticker'] = file_
    list_.append(df)
frame = pd.concat(list_)


In [4]:
#Clean up text to keep ticker
frame['ticker'] = frame['ticker'].apply(lambda x: x.replace('/home/nyao111/266/FinalProject/price_history/', "").replace(".csv", ""))

In [5]:
frame.head()

,Date,Open,High,Low,Close,Volume,Adj Close,ticker
0,2013-02-01,49.97,50.46,49.75,50.10,415900,50.10,KRC
1,2013-01-31,50.13,50.29,49.10,49.90,936300,49.90,KRC
2,2013-01-30,50.10,50.29,49.68,49.98,419100,49.98,KRC
3,2013-01-29,49.97,50.34,49.81,50.29,252100,50.29,KRC
4,2013-01-28,50.09,50.16,49.67,49.97,411300,49.97,KRC


In [6]:
#Calculate Return
frame = frame.sort(['ticker', 'Date'], ascending=[1, 1])
frame["return"] = (frame["Adj Close"].diff(1)/frame["Adj Close"].shift(1))*float(100)

/home/nyao111/.local/share/jupyter/kernels/M-learn/env/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [7]:
frame["Before Market"] = float(100)*(frame.Open - frame.Close.shift(1))/frame.Close.shift(1)
frame["During Market"] = float(100)*(frame.Close - frame.Close.shift(1))/frame.Close.shift(1)
frame["After Market"] = float(100)*(frame.Open.shift(-1) - frame.Close)/frame.Close

In [8]:
frame['Date'] = pd.to_datetime(frame['Date'])

In [9]:
frame.loc[frame["ticker"] =='AA', :].head()
#frame.loc[(frame["ticker"] =='AA' and frame['Date'] == datetime.date(2002, 7, 11)), :]

,Date,Open,High,Low,Close,Volume,Adj Close,ticker,return,Before Market,During Market,After Market
12859,1962-01-02,65.37,65.75,65.37,65.37,134400,0.67,AA,-98.520645,44.336498,44.336498,0.000000
12858,1962-01-03,65.37,66.37,65.25,66.37,179200,0.68,AA,1.492537,0.000000,1.529754,0.000000
12857,1962-01-04,66.37,66.87,66.37,66.37,193600,0.68,AA,0.000000,0.000000,0.000000,0.000000
12856,1962-01-05,66.37,66.75,66.12,66.25,169600,0.68,AA,0.000000,0.000000,-0.180805,-0.377358
12855,1962-01-08,66.00,66.00,63.50,64.00,225600,0.66,AA,-2.941176,-0.377358,-3.396226,0.000000


In [10]:
frame.head()

,Date,Open,High,Low,Close,Volume,Adj Close,ticker,return,Before Market,During Market,After Market
3320,1999-11-18,45.50,50.00,40.00,44.00,44739900,41.05,A,NaN,NaN,NaN,-2.409091
3319,1999-11-19,42.94,43.00,39.81,40.38,10897100,37.67,A,-8.233861,-2.409091,-8.227273,2.303120
3318,1999-11-22,41.31,44.00,40.06,44.00,4705200,41.05,A,8.972657,2.303120,8.964834,-3.409091
3317,1999-11-23,42.50,43.63,40.00,40.00,4274400,37.32,A,-9.086480,-3.409091,-9.090909,0.325000
3316,1999-11-24,40.13,41.94,40.00,41.06,3464400,38.30,A,2.625938,0.325000,2.650000,-0.438383


In [13]:
datapath = r'/home/nyao111/266/FinalProject/Data/*.csv' 

allFiles = glob.glob(datapath)
data = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    df = df.rename(columns = {'date':'Date'})
    df['ticker'] = file_
    list_.append(df)
data = pd.concat(list_)

### Merge with Price / timestamp

In [19]:
import datetime
test=data
test['timestamp'] = pd.to_datetime(test['timestamp'], format='%Y-%m-%d', errors='ignore')
test['Date'] = pd.to_datetime(test['Date'])

test['release_time_type'] = test['timestamp']
test['release_time_type'] = test.release_time_type.apply(lambda x: 1 if x < \
                                                         datetime.datetime(x.year, x.month, x.day,9,30,0) else x)
test['release_time_type'] = test.release_time_type.apply(lambda x: 2 if x not in [1] \
                                                         and x > datetime.datetime(x.year, x.month, x.day,9,30,0) \
                                                         and x <= datetime.datetime(x.year, x.month, x.day,16,0,0) \
                                                         else x)
test['release_time_type'] = test.release_time_type.apply(lambda x: 3 if x not in [1,2] \
                                                         and x > datetime.datetime(x.year, x.month, x.day,16,0,0) else x)

In [28]:
test['ticker']=test['ticker'].apply(lambda x: x.replace('/home/nyao111/266/FinalProject/Data/', "").replace(".csv", ""))

In [29]:
#test.dtypes
test.head(3)

,Date,timestamp,ticker,bow,items,text,orig_file,release_time_type
0,2002-07-24,2002-07-24 21:39:56,KRC,"Counter({'kilroy': 28, 'realty': 25, 'forward-...",[''],"\nTEXT:\nOn July 24, 2002 the Company issued a...",KRC/KRC-8K-20020724213956.txt.gz,3
1,2002-08-12,2002-08-12 20:56:04,KRC,"Counter({'quarterly': 8, 'report': 7, 'section...",[''],"\n\nTEXT:\nOn August 12, 2002, the Registrant ...",KRC/KRC-8K-20020812205604.txt.gz,3
2,2002-10-28,2002-10-28 21:06:49,KRC,"Counter({'kilroy': 29, 'realty': 26, 'forward-...",[''],"\n\nTEXT:\nOn October 28, 2002 the Company iss...",KRC/KRC-8K-20021028210649.txt.gz,3


In [31]:
combined = pd.merge(test, frame, how='left', on=['ticker','Date'])

In [32]:
combined['stock_performance'] = 0
combined['stock_performance'] = \
  combined.release_time_type.apply(lambda x: 1 if x ==1 else 0)*combined['Before Market']\
+ combined.release_time_type.apply(lambda x: 1 if x ==2 else 0)*combined['During Market']\
+ combined.release_time_type.apply(lambda x: 1 if x ==3 else 0)*combined['After Market']

In [35]:
combined.head(5)

,Date,timestamp,ticker,bow,items,text,orig_file,release_time_type,Open,High,Low,Close,Volume,Adj Close,return,Before Market,During Market,After Market,stock_performance
0,2002-07-24,2002-07-24 21:39:56,KRC,"Counter({'kilroy': 28, 'realty': 25, 'forward-...",[''],"\nTEXT:\nOn July 24, 2002 the Company issued a...",KRC/KRC-8K-20020724213956.txt.gz,3,22.00,22.80,21.12,22.17,236700,13.34,0.527506,-0.226757,0.544218,0.360848,0.360848
1,2002-08-12,2002-08-12 20:56:04,KRC,"Counter({'quarterly': 8, 'report': 7, 'section...",[''],"\n\nTEXT:\nOn August 12, 2002, the Registrant ...",KRC/KRC-8K-20020812205604.txt.gz,3,22.77,23.50,22.58,23.41,40200,14.08,1.881331,-0.870701,1.915542,0.384451,0.384451
2,2002-10-28,2002-10-28 21:06:49,KRC,"Counter({'kilroy': 29, 'realty': 26, 'forward-...",[''],"\n\nTEXT:\nOn October 28, 2002 the Company iss...",KRC/KRC-8K-20021028210649.txt.gz,3,21.68,21.68,21.35,21.50,70300,13.21,0.000000,0.837209,0.000000,0.000000,0.000000
3,2003-02-03,2003-02-03 21:57:04,KRC,"Counter({'contents': 29, 'table': 29, 'kilroy'...",[''],"\n\nTEXT:\nOn February 3, 2003 the Company iss...",KRC/KRC-8K-20030203215704.txt.gz,3,21.84,21.84,21.59,21.70,230700,13.61,-0.656934,-0.045767,-0.686499,-0.230415,-0.230415
4,2003-04-29,2003-04-29 08:24:19,KRC,"Counter({""company's"": 24, 'kilroy': 14, 'ffo':...",['Regulation FD Disclosure'],\n\nTEXT:\nIn accordance with SEC Release No. ...,KRC/KRC-8K-20030429082419.txt.gz,1,24.04,24.35,23.98,24.27,407300,15.55,1.236979,0.333890,1.293823,1.771735,0.333890


In [ ]:
import pickle 
pickle.dump(combined, open("/home/nyao111/266/FinalProject/Combined_fulldata", "rb"))

In [2]:
import pickle
combined=pickle.load(open("/home/nyao111/266/FinalProject/Combined_fulldata", "rb"))

In [12]:
combined.head(3)

,Date,timestamp,ticker,bow,items,text,orig_file,release_time_type,Open,High,Low,Close,Volume,Adj Close,return,Before Market,During Market,After Market,stock_performance
0,2002-07-24,2002-07-24 21:39:56,KRC,"Counter({'kilroy': 28, 'realty': 25, 'forward-...",[''],"\nTEXT:\nOn July 24, 2002 the Company issued a...",KRC/KRC-8K-20020724213956.txt.gz,3,22.00,22.80,21.12,22.17,236700,13.34,0.527506,-0.226757,0.544218,0.360848,0.360848
1,2002-08-12,2002-08-12 20:56:04,KRC,"Counter({'quarterly': 8, 'report': 7, 'section...",[''],"\n\nTEXT:\nOn August 12, 2002, the Registrant ...",KRC/KRC-8K-20020812205604.txt.gz,3,22.77,23.50,22.58,23.41,40200,14.08,1.881331,-0.870701,1.915542,0.384451,0.384451
2,2002-10-28,2002-10-28 21:06:49,KRC,"Counter({'kilroy': 29, 'realty': 26, 'forward-...",[''],"\n\nTEXT:\nOn October 28, 2002 the Company iss...",KRC/KRC-8K-20021028210649.txt.gz,3,21.68,21.68,21.35,21.50,70300,13.21,0.000000,0.837209,0.000000,0.000000,0.000000


In [3]:
import EPS_Parser as ep
reload(ep)

#Read in all EPS data; update the path based on your directory setup
path = r"/home/nyao111/266/FinalProject/EPS_parsed"
allFiles = glob.glob(path + "/*.csv")
print len(allFiles)
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
EPS = pd.concat(list_)

#Rename symbol to ticker
EPS = EPS.rename(columns = {'Symbol':'ticker'})

2543


In [4]:
EPS["Date"]=pd.to_datetime(EPS["Date"])

In [5]:
combined_w_eps = pd.merge(EPS, combined, how='inner', on=['ticker','Date'])
combined_w_eps.head(3)
print combined_w_eps.shape

(37503, 24)


In [6]:
combined_w_eps.head(3)

,Unnamed: 0,Company,ticker,Surprise,Reported_EPS,Consensus_EPS,Date,timestamp,bow,items,...,High,Low,Close,Volume,Adj Close,return,Before Market,During Market,After Market,stock_performance
0,2,Accenture plc. Class A Ordinary,ACN,3.08,0.67,0.65,2009-12-17,2009-12-17 16:16:34,"Counter({'quarter': 36, 'first': 29, 'fiscal':...",['Results of Operations and Financial Conditio...,...,42.00,41.27,41.66,5040600,39.20,-0.431801,-0.693117,-0.430210,-2.184349,-2.184349
1,3,Actuant Corporation Common Stoc,ATU,25.00,0.20,0.16,2009-12-17,2009-12-17 08:00:27,"Counter({'quarter': 30, 'first': 24, 'million'...",['Results of Operations and Financial Conditio...,...,18.65,17.03,17.43,1042500,17.34,-0.970874,4.318182,-0.965909,2.180149,4.318182
2,6,"Darden Restaurants, Inc. Common",DRI,2.38,0.43,0.42,2009-12-17,2009-12-17 17:15:50,"Counter({'sales': 31, 'fiscal': 22, 'quarter':...",['Results of Operations and Financial Conditio...,...,33.25,32.67,32.75,2276700,29.40,-1.836394,-0.959233,-1.828537,0.793893,0.793893


In [12]:
combined.shape

(195140, 19)

### Load previously-saved data


In [16]:
data=pickle.load(open("/home/nyao111/266/FinalProject/Fulldata_combined_w_eps"))
market_data=pickle.load(open("/home/nyao111/266/FinalProject/market_data"))

In [17]:
##########Drop Columns not used#########
data = data.drop(['Before Market', 'During Market', 'After Market','High','Low','Close','Volume',\
                  'Open', 'Adj Close'], axis=1)

#########Combine stock return with market return
combined = pd.merge(data, market_data[['Date', 'Market Before', 'Market During', 'Market After' ]],\
                    how='left', on=['Date'])

#########Select market return based on report release time####
combined['market_performance'] = 0
combined['market_performance'] = \
  combined.release_time_type.apply(lambda x: 1 if x ==1 else 0)*combined['Market Before']\
+ combined.release_time_type.apply(lambda x: 1 if x ==2 else 0)*combined['Market During']\
+ combined.release_time_type.apply(lambda x: 1 if x ==3 else 0)*combined['Market After']

combined = combined.drop(['Market Before', 'Market During', 'Market After'], axis=1)


#Finally, create the label for our model.  A normalized return by taking the difference
#between stock return and market return.  If the difference >  1%: UP
#                                                           < -1%: DOWN
#                                                            else: STAY
############################################################################################################
combined['normalized_performance'] = combined.stock_performance - combined.market_performance
combined['label'] = combined.normalized_performance
combined['label'] = combined.label.apply(lambda x: 'UP' if x >1 else x)
combined['label'] = combined.label.apply(lambda x: 'DOWN' if (isinstance(x, str) == 0 and x <-1) else x)
combined['label'] = combined.label.apply(lambda x: x if isinstance(x, str) else 'STAY')

In [19]:
combined.head()

,Unnamed: 0,Company,ticker,Surprise,Reported_EPS,Consensus_EPS,Date,timestamp,bow,items,text,orig_file,release_time_type,return,stock_performance,market_performance,normalized_performance,label
0,2,Accenture plc. Class A Ordinary,ACN,3.08,0.67,0.65,2009-12-17,2009-12-17 16:16:34,"Counter({'quarter': 36, 'first': 29, 'fiscal':...",['Results of Operations and Financial Conditio...,\n\nTEXT:\nCheck the appropriate box below if ...,ACN/ACN-8K-20091217161634.txt.gz,3,-0.431801,-2.184349,0.162397,-2.346746,DOWN
1,3,Actuant Corporation Common Stoc,ATU,25.00,0.20,0.16,2009-12-17,2009-12-17 08:00:27,"Counter({'quarter': 30, 'first': 24, 'million'...",['Results of Operations and Financial Conditio...,\n\nTEXT:\nUNITED STATES\nSECURITIES AND EXCHA...,ATU/ATU-8K-20091217080027.txt.gz,1,-0.970874,4.318182,-0.254242,4.572424,UP
2,6,"Darden Restaurants, Inc. Common",DRI,2.38,0.43,0.42,2009-12-17,2009-12-17 17:15:50,"Counter({'sales': 31, 'fiscal': 22, 'quarter':...",['Results of Operations and Financial Conditio...,\n\nTEXT:\nCheck the appropriate box below if ...,DRI/DRI-8K-20091217171550.txt.gz,3,-1.836394,0.793893,0.162397,0.631496,STAY
3,8,FedEx Corporation Common Stock,FDX,3.77,1.10,1.06,2009-12-17,2009-12-17 08:02:09,"Counter({'fedex': 33, 'fuel': 11, 'second': 10...",['Results of Operations and Financial Conditio...,\n\nTEXT:\nCheck the appropriate box below if ...,FDX/FDX-8K-20091217080209.txt.gz,1,-6.090106,-3.846581,-0.254242,-3.592340,DOWN
4,9,"General Mills, Inc. Common Stoc",GIS,6.94,0.77,0.72,2009-12-17,2009-12-17 08:16:14,"Counter({'net': 38, 'sales': 37, 'percent': 36...",['Results of Operations and Financial Conditio...,"\n\nTEXT:\nRegistrant's telephone number, incl...",GIS/GIS-8K-20091217081614.txt.gz,1,1.101393,0.893249,-0.254242,1.147491,UP


In [20]:
pickle.dump(combined, open("/home/nyao111/266/FinalProject/Fulldata_wY", 'wb'))

In [21]:
combined_w_eps.keys()

Index([u'Unnamed: 0', u'Company', u'ticker', u'Surprise', u'Reported_EPS',
       u'Consensus_EPS', u'Date', u'timestamp', u'bow', u'items', u'text',
       u'orig_file', u'release_time_type', u'Open', u'High', u'Low', u'Close',
       u'Volume', u'Adj Close', u'return', u'Before Market', u'During Market',
       u'After Market', u'stock_performance'],
      dtype='object')

### Making count vectors

In [15]:
combined_w_eps.keys()

Index([u'Unnamed: 0', u'Company', u'ticker', u'Surprise', u'Reported_EPS',
       u'Consensus_EPS', u'Date', u'timestamp', u'bow', u'items', u'text',
       u'orig_file', u'release_time_type', u'Open', u'High', u'Low', u'Close',
       u'Volume', u'Adj Close', u'return', u'Before Market', u'During Market',
       u'After Market', u'stock_performance'],
      dtype='object')

In [16]:
numeric=combined_w_eps.drop(["bow", 'text'], axis=1)
numeric.shape

(37503, 22)

In [48]:
from collections import Counter
example=eval(combined_w_eps.loc[ 2:5, 'bow'])
len(example)

578

In [49]:
bow_vector=vectorizer1.fit_transform(example)

In [17]:
vectorizer1=CountVectorizer(stop_words='english')
text_vector=vectorizer1.fit_transform(combined_w_eps["text"])

In [21]:
print  text_vector.shape

(37503, 140874)


In [ ]:
## Non-negative factorization of the vectors, may need to try more
from sklearn.decomposition import NMF
model = NMF(n_components=5000, init='random', random_state=0)
text_nmf = model.fit(text_vector)

In [18]:
pickle.dump(text_vector, open("Text_CountVec_eps", 'wb'))
pickle.dump(combined_w_eps, open("Fulldata_combined_w_eps", "wb"))
pickle.dump(numeric,open("NumericData", "wb") )